# Train

In [2]:
# Convert to .py when done
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import joblib

import warnings
warnings.filterwarnings("ignore")
import os

In [3]:
'''
Delete for .py file 
'''
# Construct the path to move up two directory levels
higher_dir = os.path.abspath('../../')
os.chdir(higher_dir)

from module.config import *

user_data_dir_train = PROJ_ROOT/DATA_DIR/'raw'/'user_data'/'train.csv'
user_data_dir_pred = PROJ_ROOT/DATA_DIR/'raw'/'user_data'/'pred.csv'
synth_data_dir_train = PROJ_ROOT/DATA_DIR/'raw'/'synth_data_train.csv'
synth_data_dir_pred = PROJ_ROOT/DATA_DIR/'raw'/'synth_data_pred.csv'
standardized_data_train = PROJ_ROOT/DATA_DIR/'interim'/'train_data_standardized.csv'
standardized_data_pred = PROJ_ROOT/DATA_DIR/'interim'/'pred_data_standardized.csv'

# Check if train.csv and pred.csv exist in user_data folder
if os.path.exists(user_data_dir_train) and os.path.exists(user_data_dir_pred):
    train_df = pd.read_csv(user_data_dir_train, sep = '\t')
    pred_df = pd.read_csv(user_data_dir_pred, sep = '\t')
else:
    train_df = pd.read_csv(synth_data_dir_train, sep = '\t')
    pred_df = pd.read_csv(synth_data_dir_pred, sep = '\t')

In [4]:
print(os.getcwd())

C:\Users\MAMM01\OneDrive - ROC Mondriaan\Documenten\Workspace\CEDA\GitHub\Uitnodigingsregel


In [ ]:
# Random Forest Regressor model
def randomforestregressormodel_train(dataset_train, random_state = random_seed):
    X = dataset_train.drop("Dropout", axis=1).values
    y = dataset_train.Dropout.values
    rf = RandomForestRegressor(random_state=random_seed)

    rf_gridsearch = GridSearchCV(rf, rf_parameters, n_jobs = -1, verbose = 2)
    rf_gridsearch.fit(X, y) 
    best_params = rf_gridsearch.best_params_
    best_rf_model = RandomForestRegressor(**best_params)
    best_rf_model.fit(X, y) 
    joblib.dump(best_rf_model, 'models/random_forest_regressor.joblib')
    return best_rf_model


In [ ]:
best_rf_model = randomforestregressormodel_train (train_df)

In [ ]:
train_df_sdd = pd.read_csv(standardized_data_train, sep = '\t')

In [ ]:
def lassoregressionmodel_train (dataset_train_sdd): 
    X = dataset_train_sdd.drop("Dropout", axis=1)
    y = dataset_train_sdd.Dropout.values
    lasso_model = Lasso(random_state = random_seed)

    # score base on r2
    param = {'alpha': alpha_range}
    lasso_grid_search = GridSearchCV(lasso_model, param_grid = param, cv=5, n_jobs = -1, verbose = 2)
    lasso_grid_search.fit(X, y)

    best_params = lasso_grid_search.best_params_
    best_lasso_model = Lasso(**best_params)
    best_lasso_model.fit(X, y) 
    joblib.dump(best_lasso_model, 'models/lasso_regression.joblib')
    return best_lasso_model

In [ ]:
best_lasso = lassoregressionmodel_train (train_df_sdd)

In [35]:
def supportvectormachinemodel_train(dataset_train):
    X = dataset_train.drop("Dropout", axis=1).values
    y = dataset_train.Dropout.values
    
    svm_gridsearch = GridSearchCV(SVC(random_state=random_seed, probability = True), svm_parameters, refit = True, n_jobs = -1, verbose = 2) 
    svm_gridsearch.fit(X, y)

    best_params = svm_gridsearch.best_params_
    best_svm_model = SVC(**best_params, probability = True)
    best_svm_model.fit(X, y) 
    #joblib.dump(best_svm_model, 'models/support_vector_machine.joblib')
    return best_svm_model

In [37]:
best_svm = supportvectormachinemodel_train(train_df)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


In [38]:
test_set = train_df.drop("Dropout", axis=1).values
probalities = best_svm.predict_proba(test_set)

In [41]:
probalities

array([[0.09795643, 0.90204357],
       [0.09795643, 0.90204357],
       [0.09795643, 0.90204357],
       ...,
       [0.08459558, 0.91540442],
       [0.08459558, 0.91540442],
       [0.08459558, 0.91540442]])